# Heroes of Pymoli Data Analysis

* OBSERVED TREND 1:  Over 80% of the players are males.  This could indicate that males are more likely to play and purchase the games.  However this could also indicate that the games themselves are designed to marketed more toward males that females.
* OBSERVED TREND 2:  Over 65% of revenue is generated by players in the 15-24 year old segments of Age Groups.  It may be worth developing more items geared toward this age group to generate more sales.
* OBSERVED TREND 3:  Even though the 30-34 and the Under 10 year old Age Groups do not have the largest counts of purchases the data indicates that these two demographics are worth looking at.  They both purchase the largest Average Purchases.  The 30-34 group is probably more established career-wise and have more expendable income than the younger groups with which to purchase recreational games.  The under 10 year olds have parents that are willing to spend money on their entertainment.  It may be worth looking into developing games to target these two groups more in order to drive future sales.

In [98]:
import pandas as pd
import os

In [99]:
df = pd.read_json("purchase_data.json.txt")

### Player Count

In [100]:
df.describe(include = 'all')
totalPlayers = df['SN'].nunique()
playerCount = pd.DataFrame({'Total Players':totalPlayers},index=[0])
playerCount

,Total Players
0,573


### Purchasing Analysis (Total)

In [101]:
itemIds = df['Item ID'].nunique()
avgPrice = df['Price'].unique().mean()
totalPurchases = len(df)
totalRev = df['Price'].sum()
PurchaseAnalysis = pd.DataFrame({'Number of Unique Items':totalPlayers,'Average Price':"${:,.2f}".format(avgPrice),'Number of Purchases':totalPurchases,'Total Revenue':"${:,.2f}".format(totalRev)},index=[0])
PurchaseAnalysis = PurchaseAnalysis[['Number of Unique Items', 'Average Price','Number of Purchases','Total Revenue']]
PurchaseAnalysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,573,$2.97,780,"$2,286.33"


### Gender Demogaphics

In [102]:
Names = pd.DataFrame(df.groupby('Gender').SN.nunique())
Names.reset_index(inplace = True)
Percent = pd.DataFrame(df.groupby('Gender').SN.nunique()/totalPlayers)
Percent.reset_index(inplace=True)
merge_table = pd.merge(Percent, Names, on=['Gender'], how="inner")
merge_table.columns = ['', 'Percentage of Players','Total Count']
merge_table['Percentage of Players'] = pd.Series(["{0:.2f}%".format(val * 100) for val in merge_table['Percentage of Players']], index = merge_table.index)
merge_table

,,Percentage of Players,Total Count
0,Female,17.45%,100
1,Male,81.15%,465
2,Other / Non-Disclosed,1.40%,8


### Purchasing Analysis (Gender)

In [117]:
All = pd.DataFrame({'count' : df.groupby( [ "Gender", "Price"] ).size()}).reset_index()
#All
Female = pd.DataFrame(All[All['Gender'] == 'Female'])
Female['Total Purchase Value'] = Female['Price']*Female['count']
Female2 = pd.DataFrame({'Gender':'Female','Purchase Count':Female['count'].sum(),'Average Purchase Price':"${:,.2f}".format(Female['Price'].mean()),'Total Purchase Value':"${:,.2f}".format(Female['Total Purchase Value'].sum()),'Normalized Totals':"${:,.2f}".format(Female['Price'].mean())},index=[0])
Female2

male = pd.DataFrame(All[All['Gender'] == 'Male'])
male['Total Purchase Value'] = male['Price']*male['count']
male2 = pd.DataFrame({'Gender':'Male','Purchase Count':male['count'].sum(),'Average Purchase Price':"${:,.2f}".format(male['Price'].mean()),'Total Purchase Value':"${:,.2f}".format(male['Total Purchase Value'].sum()),'Normalized Totals':"${:,.2f}".format(male['Price'].mean())},index=[0])
male2

other = pd.DataFrame(All[All['Gender'] == 'Other / Non-Disclosed'])
other['Total Purchase Value'] = other['Price']*other['count']
other2 = pd.DataFrame({'Gender':'Other / Non-Disclosed','Purchase Count':other['count'].sum(),'Average Purchase Price':"${:,.2f}".format(other['Price'].mean()),'Total Purchase Value':"${:,.2f}".format(other['Total Purchase Value'].sum()),'Normalized Totals':"${:,.2f}".format(other['Price'].mean())},index=[0])
other2

verticalStack = pd.concat([Female2, male2, other2], axis=0)
verticalStack = verticalStack[['Gender', 'Purchase Count','Average Purchase Price','Total Purchase Value','Normalized Totals']]
verticalStack

,Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
0,Female,136,$2.88,$382.91,$2.88
0,Male,633,$2.99,"$1,867.68",$2.99
0,Other / Non-Disclosed,11,$3.25,$35.74,$3.25


### Age Demographics

In [105]:
labels = ["Age", "SN"]
ages = df.loc[:,labels]
ages = ages.drop_duplicates(subset = ["Age", "SN"])

bins = [0,10,15,20,25,30,35,40,400]

group_labels = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

pd.cut(ages['Age'], bins, labels=group_labels)
ages['Age Group'] = pd.cut(ages['Age'], bins, labels=group_labels)
ages = ages.drop('SN', 1)
groupedAges =  ages.groupby('Age Group')
groupedAges.head()
totals = groupedAges.agg(['count'])
totals.columns = totals.columns.droplevel()
totals['Percentage of Players'] = pd.Series(["{0:.2f}%".format(val * 100) for val in totals['count']/len(ages)],index = totals.index)
totals = totals.rename(columns={'count': 'Total Count'})
totals = totals[['Percentage of Players','Total Count']]
totals

,Percentage of Players,Total Count
Age Group,,
<10,3.84%,22
10-14,9.42%,54
15-19,24.26%,139
20-24,40.84%,234
25-29,9.08%,52
30-34,7.68%,44
35-39,4.36%,25
40+,0.52%,3


### Purchasing  Analysis (Age)

In [122]:
labels = ["Age", "SN","Price"]
ages = df.loc[:,labels]
ages = ages.drop_duplicates(subset = ["Age", "SN"])

bins = [0,10,15,20,25,30,35,40,400]

group_labels = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

pd.cut(ages['Age'], bins, labels=group_labels)
ages['Age Group'] = pd.cut(ages['Age'], bins, labels=group_labels)
del ages['SN']
ages
groupedAges =  ages.groupby('Age Group')
groupedAges.head()
totals = groupedAges.agg(['count','mean','sum'])
totals.drop(('Age', 'mean'), axis = 1, inplace = True)
totals.drop(('Age', 'sum'), axis = 1, inplace = True)
totals.drop(('Price', 'count'), axis = 1, inplace = True)
totals.columns = totals.columns.droplevel()
totals = totals.rename(columns={'count': 'Purchase Count','mean':'Average Purchase Price','sum':'Total Purchase Value'})
totals['Average Purchase Price'] = totals['Average Purchase Price'].map('${:,.2f}'.format)
totals['Total Purchase Value'] = totals['Total Purchase Value'].map('${:,.2f}'.format)
totals['Normalized Totals'] = totals['Average Purchase Price']
totals

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Group,,,,
<10,22,$3.16,$69.50,$3.16
10-14,54,$2.89,$155.94,$2.89
15-19,139,$2.90,$403.12,$2.90
20-24,234,$2.99,$699.12,$2.99
25-29,52,$2.92,$151.65,$2.92
30-34,44,$3.33,$146.48,$3.33
35-39,25,$2.87,$71.64,$2.87
40+,3,$2.88,$8.64,$2.88


### Top Spenders

In [112]:
count = df.groupby('SN')['Price'].count()
count
sums = df.groupby('SN')['Price'].sum()
sums
avgs = df.groupby('SN')['Price'].mean()
avgs
summary_table = pd.DataFrame({'Purchase Count':count,'Average Purchase Price':avgs,'Total Purchase Value':sums})
summary_table = summary_table.sort_values('Total Purchase Value',ascending=False)
summary_table['Average Purchase Price'] = summary_table['Average Purchase Price'].map('${:,.2f}'.format)
summary_table['Total Purchase Value'] = summary_table['Total Purchase Value'].map('${:,.2f}'.format)
summary_table = summary_table[['Purchase Count','Average Purchase Price','Total Purchase Value']]
summary_table.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


### Most Popular Items

In [113]:
count = df.groupby(['Item ID','Item Name'])['Price'].count()
count
sums = df.groupby(['Item ID','Item Name'])['Price'].sum()
sums
avgs = df.groupby(['Item ID','Item Name'])['Price'].mean()
avgs
summary_table = pd.DataFrame({'Purchase Count':count,'Item Price':avgs,'Total Purchase Value':sums})
summary_table['Item Price'] = summary_table['Item Price'].map('${:,.2f}'.format)
summary_table['Total Purchase Value'] = summary_table['Total Purchase Value'].map('${:,.2f}'.format)
summary_table = summary_table[['Purchase Count','Item Price','Total Purchase Value']]
summary_table = summary_table.sort_values('Purchase Count',ascending=False)
summary_table.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41


### Most Profitable Items

In [96]:
count = df.groupby(['Item ID','Item Name'])['Price'].count()
count
sums = df.groupby(['Item ID','Item Name'])['Price'].sum()
sums
avgs = df.groupby(['Item ID','Item Name'])['Price'].mean()
avgs
summary_table = pd.DataFrame({'Purchase Count':count,'Item Price':avgs,'Total Purchase Value':sums})
summary_table = summary_table.sort_values('Total Purchase Value',ascending=False)
summary_table['Item Price'] = summary_table['Item Price'].map('${:,.2f}'.format)
summary_table['Total Purchase Value'] = summary_table['Total Purchase Value'].map('${:,.2f}'.format)
summary_table = summary_table[['Purchase Count','Item Price','Total Purchase Value']]
summary_table.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
